## Regresión

Para el problema de pronóstico, elegimos el dataset de [Air Pollution Forecasting](https://www.kaggle.com/datasets/rupakroy/lstm-datasets-multivariate-univariate?resource=download)

Este conjunto de datos informa sobre el clima y el nivel de contaminación cada hora durante cinco años en la embajada de Estados Unidos en Pekín, China.

Los datos incluyen la fecha y la hora, la concentración de PM2.5 y la información meteorológica, como el punto de rocío, la temperatura, la presión, la dirección y la velocidad del viento, y el número acumulado de horas de nieve y lluvia. El objetivo es predecir el nivel de contaminación de aire, que es el PM2.5

La lista completa de características de los datos es la siguiente:

- `date`: La fecha del registro (es por hora)
- `pollution`: concentración de PM2.5 (variable objetivo)
- `dew`: punto de rocío
- `temp`: temperatura
- `press`: presión
- `wnd_dir`: dirección del viento combinada
- `wnd_spd`: velocidad del viento acumulada
- `snow`: horas de nieve acumuladas
- `rain`: horas de lluvia acumuladas

Nuestro flujo de trabajo será el siguiente:
+ EDA
+ Entrenamiento y selección de un modelo de pronóstico lineal clásico para el caso univariado
+ Evaluación del modelo clásico
+ Selección, entrenamiento y validación de distintos modelos basados en redes neuronales en el caso univariado
+ Selección, entrenamiento y validación de distintos modelos basados en redes neuronales en el caso multivariado
+ Comparación de los modelos de redes neuronales univariados vs los multivariados
+ Comparación del modelo clásico univariado contra las redes neuronales univariadas
+ Selección del mejor modelo general y optimizarlo con `Optuna`

Para el registro y comparación de modelos usaremos MLFlow

#### Cargar los datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
from scipy.stats import boxcox
import statsmodels as st
from statsmodels.tsa.seasonal import MSTL
from sklearn.preprocessing import power_transform
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy.stats import normaltest
from typing import List, Optional, Tuple
import itertools
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.special import inv_boxcox
import scipy.stats as stats
import dagshub
import mlflow
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import joblib

: 

In [2]:
class LinearForecast:
    def __init__(self, data: Optional[pd.DataFrame] = None):
        self.data = data
        self.results = None  # Holds the SARIMAX fitted model if needed

    def read_data(self, data_path: str = None, data: pd.DataFrame = None, y_col: str = "y", date_col: str = "date"):
        """
        Reads data from an Excel file or directly from a DataFrame, sets the date column as index, and renames the target column to 'y'.

        Parameters:
        - data_path (str): The path to the Excel file (optional if data is provided).
        - data (pd.DataFrame): Pre-loaded data as a DataFrame (optional if data_path is provided).
        - y_col (str): The name of the target column to rename to 'y'.
        - date_col (str): The name of the column to set as the index.

        Returns:
        - pd.DataFrame: The processed DataFrame.
        """
        if data is not None:
            self.data = data
        elif data_path is not None:
            self.data = pd.read_excel(data_path)
        else:
            raise ValueError("Either data_path or data must be provided.")

        self.data = self.data.set_index(date_col)
        self.data = self.data.rename(columns={y_col: "y"})
        return self.data


    def plot_acf_pacf(self, data=None, kwargs=None):
        """
        Plots the ACF and PACF of a given time series.
        If no data is provided, it uses the 'y' column from self.data.
        """
        if kwargs is None:
            kwargs = {}

        data = data if data is not None else self.data["y"]

        f = plt.figure(figsize=(8, 5))
        ax1 = f.add_subplot(121)
        plot_acf(data, zero=False, ax=ax1, **kwargs)

        ax2 = f.add_subplot(122)
        plot_pacf(data, zero=False, ax=ax2, method="ols", **kwargs)
        plt.show()

    def adf_test(self, timeseries=None):
        """
        Performs the Augmented Dickey-Fuller test to check stationarity of a given time series.
        If no timeseries is provided, it uses self.data['y'].
        """
        timeseries = timeseries if timeseries is not None else self.data["y"]

        print("Results of Dickey-Fuller Test:")
        dftest = adfuller(timeseries, autolag="AIC")
        dfoutput = pd.Series(dftest[0:4],
                             index=["Test Statistic", "p-value", "#Lags Used", "Number of Observations Used"])

        for key, value in dftest[4].items():
            dfoutput["Critical Value (%s)" % key] = value
        print(dfoutput)

        if (dftest[1] <= 0.05) & (dftest[4]["5%"] > dftest[0]):
            print("\u001b[32mStationary\u001b[0m")
        else:
            print("\x1b[31mNon-stationary\x1b[0m")

    def mstl_descomposition(self, periods_seasonality, stl_kwargs=None):
        """
        Performs an MSTL decomposition on the 'y' column of self.data.
        Returns the decomposition results and also plots them.
        """
        if stl_kwargs is None:
            stl_kwargs = {"seasonal_deg": 0}

        model = MSTL(self.data["y"], periods=periods_seasonality, stl_kwargs=stl_kwargs)
        res = model.fit()

        fig, ax = plt.subplots(3 + len(periods_seasonality), 1, sharex=True, figsize=(8, 8))
        res.observed.plot(ax=ax[0])
        ax[0].set_ylabel("Observed")
        res.trend.plot(ax=ax[1])
        ax[1].set_ylabel("Trend")

        for i, s_period in enumerate(periods_seasonality):
            res.seasonal[f"seasonal_{s_period}"].plot(ax=ax[i + 2])
            ax[i + 2].set_ylabel(f"seasonal_{s_period}")

        res.resid.plot(ax=ax[2 + len(periods_seasonality)])
        ax[2 + len(periods_seasonality)].set_ylabel("Residual")

        fig.tight_layout()
        return res

    def plot_time_series(self, figsize=(10, 5)):
        """
        Plots the main 'y' time series.
        """
        f = plt.figure(figsize=figsize)
        ax = f.add_subplot(111)
        ax.plot(self.data["y"])
        plt.title("Time Series Plot")
        plt.show()

    def plot_boxp(self, data=None, figsize=(10, 5)):
        """
        Plots a boxplot for the provided DataFrame or for self.data if none is provided.
        Ensures the data is converted to a DataFrame and index is reset to avoid issues with time series indexing.
        """
        if data is None:
            data = self.data.copy()

        # Convert to DataFrame if not already
        data_box = pd.DataFrame(data).reset_index(drop=True)

        f = plt.figure(figsize=figsize)
        data_box.boxplot()
        plt.title("Box Plot")
        plt.show()

    def timeseries_transformation(self, transformation="log"):
        """
        Applies a specified transformation to the 'y' series and plots histograms and boxplots
        for the original and transformed data. Supported transformations are 'log', 'sqrt', 'boxcox', 'yeo-johnson'.
        Box-Cox transformation is skipped if values are not strictly positive.
        """
        data = self.data["y"]
        FIG_SIZE_LARGE = (10, 5)

        def plot_histograms(data_transformed, original_title, transformed_title):
            fig, ax = plt.subplots(1, 2, figsize=FIG_SIZE_LARGE)
            data.hist(bins=50, ax=ax[0])
            ax[0].set_title(original_title)
            data_transformed.hist(bins=50, ax=ax[1])
            ax[1].set_title(transformed_title)
            plt.tight_layout()
            plt.show()

        def plot_boxplots(data_transformed, original_title, transformed_title):
            fig, ax = plt.subplots(1, 2, figsize=FIG_SIZE_LARGE)
            pd.DataFrame(data).boxplot(ax=ax[0])
            ax[0].set_title(original_title)
            pd.DataFrame(data_transformed).boxplot(ax=ax[1])
            ax[1].set_title(transformed_title)
            plt.tight_layout()
            plt.show()

        transformations = {
            "sqrt": lambda d: np.sqrt(d),
            "log": lambda d: np.log1p(d),
            "boxcox": lambda d: pd.DataFrame({
                "box_cox": power_transform(d.to_numpy().reshape(-1, 1), method="box-cox").squeeze()
            }) if (d > 0).all() else None,
            "yeo-johnson": lambda d: pd.DataFrame({
                "yeo-johnson": power_transform(d.to_numpy().reshape(-1, 1), method="yeo-johnson").squeeze()
            }),
        }

        if transformation not in transformations:
            raise ValueError(f"Unsupported transformation: {transformation}")

        if transformation == "boxcox" and not (data > 0).all():
            print("Skipping Box-Cox transformation: Data must be strictly positive.")
            return

        transformed = transformations[transformation](data)
        if transformed is None:
            print(f"Transformation {transformation} could not be applied. Check data or transformation requirements.")
            return

        plot_histograms(transformed, "Original Data", f"Transformed ({transformation.title()})")
        plot_boxplots(transformed, "Original Data", f"Transformed ({transformation.title()})")


    def timeseries_scaler(self, scaler="minmax"):
        """
        Scales the 'y' series using the specified scaler ('minmax' or 'standard').
        Returns the scaled values as a numpy array.
        """
        data = self.data["y"]

        if scaler == "minmax":
            mm_scaler = MinMaxScaler()
            scaled_values = mm_scaler.fit_transform(data.values.reshape(-1, 1))
        elif scaler == "standard":
            standard_scaler = StandardScaler()
            scaled_values = standard_scaler.fit_transform(data.values.reshape(-1, 1))
        else:
            raise ValueError(f"Scaler '{scaler}' is not valid. Choose from 'minmax' or 'standard'.")

        return scaled_values

    def grid_search(self, p: list, d: list, q: list, P: list, D: list, Q: list, X: list, trends: list,
                    start: int = 0, end: int = 10, perc: float = 0.1, data = None):
        """
        Perform a grid search for SARIMAX hyperparameters over the provided lists of (p, d, q, P, D, Q, trends).
        Only a portion (perc) of the data is used for training to speed up the search.
        """

        if data is None:
            data = self.data.copy()

        train_data = self.data[:int(perc * len(self.data))]

        no_estacional = list(itertools.product(p, d, q))
        estacional = list(itertools.product(P, D, Q, X))

        sarimax_params = list(itertools.product(no_estacional, estacional, trends))

        resultados = pd.DataFrame(columns=["params", "AIC", "BIC", "LLF"])

        for i, (non_seasonal, seasonal, trend) in enumerate(sarimax_params[start:end]):
            mod = SARIMAX(
                endog=train_data,
                trend=trend,
                order=non_seasonal,
                seasonal_order=seasonal,
            )
            results = mod.fit(disp=False)

            resultados.loc[i, "params"] = str((non_seasonal, seasonal, trend))
            resultados.loc[i, "AIC"] = results.aic
            resultados.loc[i, "BIC"] = results.bic
            resultados.loc[i, "LLF"] = results.llf

        return resultados

    def train_sarimax(self, sarimax_params: tuple, perc: float = 0.75, data=None):
        """
        Train a SARIMAX model using specified parameters:
        sarimax_params is a tuple (non_seasonal_order, seasonal_order, trend).
        """

        if data is None:
            data = self.data.copy()

        n_train = int(perc * len(self.data))
        train_data = self.data[:n_train]

        mod = SARIMAX(
            endog=train_data,
            trend=sarimax_params[2],  # Trend ('n', 'c', 'ct', etc.)
            order=sarimax_params[0],  # Non-seasonal order (p, d, q)
            seasonal_order=sarimax_params[1],  # Seasonal order (P, D, Q, m)
        )
        self.results = mod.fit(disp=False)
        print(f"AIC={self.results.aic}")
        print(f"BIC={self.results.bic}")
        print(f"Log-likelihood={self.results.llf}")
        return self.results

    def predict_sarimax(self, n_periods: int = 100, data = None, perc: float = 0.75):
        """
        Predict values using the trained SARIMAX model for the desired number of periods.
        The split is 75/25. The function returns a DataFrame with the predictions.
        """
        if data is None:
            data = self.data.copy()
        if self.results is None:
            raise ValueError("Model has not been trained yet. Please call train_sarimax first.")

        n_train = int(perc * len(self.data))
        train_data = self.data[:n_train]
        n_test = len(self.data) - n_train
        test_data = self.data[-n_test:]

        pred = self.results.predict(len(train_data), len(train_data) + len(test_data) - 1)
        pred = pred[:n_periods].to_frame(name="prediction")

        pred.plot()
        plt.legend(["Prediction"])
        plt.title("SARIMAX Prediction")
        plt.show()
        return pred

    def forecast_sarimax(self, n_periods: int = 100, col_analysis: str = "y"):
        """
        Forecast using the SARIMAX model and include confidence intervals.
        Returns a DataFrame with actuals, confidence intervals, and predicted mean.
        """
        if self.results is None:
            raise ValueError("Model has not been trained yet. Please call train_sarimax first.")

        prediction = self.results.get_prediction()
        pred = pd.concat([self.data, prediction.conf_int(), prediction.predicted_mean], axis=1)

        fig, ax = plt.subplots()
        pred[col_analysis][:n_periods].plot(style="r--", label="Actual", ax=ax)
        pred["predicted_mean"][:n_periods].plot(style="b", alpha=0.4, label="Prediction", ax=ax)
        plt.legend()
        plt.title("SARIMAX Forecast")
        plt.show()
        return pred

    def horizon_pred(self, horizon: int, num: int, col_analysis: str, order: tuple, seasonal_order: tuple):
        """
        Perform a rolling horizon prediction for a specified time period.
        horizon: number of steps to forecast
        num: how many data points from the end of self.data to use (the last 'num' points)
        col_analysis: column name for analysis (e.g., 'y')
        order: non-seasonal order tuple (p, d, q)
        seasonal_order: seasonal order tuple (P, D, Q, m)
        """
        if self.data is None or self.data.empty:
            raise ValueError("The data parameter is empty or None.")
        if col_analysis not in self.data.columns:
            raise ValueError(f"'{col_analysis}' column is missing in the input data.")

        y_train = self.data[col_analysis][-num:-horizon]
        y_test = self.data[col_analysis][-horizon:]
        y_pred = []

        for _ in range(horizon):
            mod = SARIMAX(
                endog=y_train,
                trend="ct",
                order=order,
                seasonal_order=seasonal_order,
            )
            results = mod.fit(disp=False)
            pred = results.predict(start=len(y_train), end=len(y_train))
            y_train = np.append(y_train, pred[0])
            y_pred.append(pred[0])

        pred_df = pd.DataFrame({"Actual": y_test, "Predicted": y_pred})
        pred_df.plot(title="Horizon Prediction")
        return pred_df

    def error_metric(self, y_test, y_pred):
        """
        Calculate evaluation metrics for predictions (MAPE, MAD).
        """
        if len(y_test) != len(y_pred):
            raise ValueError("Lengths of y_test and y_pred must match.")

        mape = np.mean(
            np.abs(
                (np.array(y_test) - np.array(y_pred)) / np.where(np.array(y_test) == 0, 1, np.array(y_test))
            )
        ) * 100
        mad = np.mean(np.abs(np.array(y_test) - np.array(y_pred)))

        return {"MAPE": mape, "MAD": mad}

    def consolidated_ts_df(self, data: Optional[pd.DataFrame] = None, plot_graphs: bool = True) -> pd.DataFrame:
        """
        Creates a DataFrame that consolidates multiple transformations and scalings of the 'y' series
        (log, sqrt, boxcox, yeo-johnson, min-max scaler, standard scaler).
        If no data is provided, it uses self.data.
        Optionally plots histograms of all generated columns if plot_graphs is True.
        Returns the DataFrame and the lambda value used for the Box-Cox transformation.
        """
        if data is None:
            df = self.data.copy()
        else:
            df = data.copy()

        # Ensure 'y' is present in df
        if 'y' not in df.columns:
            raise ValueError("The dataframe must have a 'y' column for transformations.")

        # Apply transformations
        df['log'] = np.log1p(df['y'])
        df['sqrt'] = np.sqrt(df['y'])

        # Box-Cox transformation (with lambda retrieval)
        if (df['y'] > 0).all():  # Box-Cox requires strictly positive values
            df['box_cox'], lambda_value = boxcox(df['y'])
        else:
            df['box_cox'] = np.nan  # Skip Box-Cox if data is not strictly positive
            lambda_value = None

        # Yeo-Johnson transformation
        df['yeo_j'] = power_transform(df['y'].to_numpy().reshape(-1, 1), method='yeo-johnson')

        # Min-Max scaling
        df['mm_scaler'] = MinMaxScaler().fit_transform(df['y'].values.reshape(-1, 1))

        # Standard scaling
        df['std_scaler'] = StandardScaler().fit_transform(df['y'].values.reshape(-1, 1))

        # Plot histograms
        if plot_graphs:
            df.hist(bins=50, figsize=(10, 8))
            plt.tight_layout()
            plt.show()

        return df, lambda_value

In [ ]:
df = pd.read_csv("data/Regression/LSTM-Multivariate_pollution.csv")
df.shape

### EDA

In [ ]:
df.head()

In [ ]:
def reporte(df):
    dtyp = pd.DataFrame(df.dtypes, columns=['Tipo'])
    missing = pd.DataFrame(df.isnull().sum(), columns=['Valores_Nulos'])
    unival = pd.DataFrame(df.nunique(), columns=['Valores_Unicos'])
    maximo = pd.DataFrame(df.max(), columns=['Max'])
    minimo = pd.DataFrame(df.min(), columns=['Min'])
    return dtyp.join(missing).join(unival).join(maximo).join(minimo)

reporte(df)

Podemos ver que los datos recolectados son desde el 2do de enero de 2010 hasta el 31 de diciembre de 2014. No hay datos nulos en ninguna de nuestras variables. Vamos a convertir nuestra columna date a datetime

In [6]:
df['date'] = pd.to_datetime(df['date'])

In [7]:
#Calculo de estadísticos básicos.
# Algunas utilerías que vamos a usar
def autolabel(rects, ax):
    """
    Método auxiliar para agregarle el númerito correspondiente a su valor
    a la barra en una gráfica de barras.

    Esta función no la hice yo (aunque sí la modifiqué). La origi está en:
    https://matplotlib.org/3.1.1/gallery/lines_bars_and_markers/barchart.html

    rects: La figura de la gráfica guardada en una variable
    ax: El eje donde se está graficando.
    """
    # attach some text labels
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2.,
                1.05*height,
                '%d'%int(height),
                ha='center', va='bottom')

def _get_colors_to_use(variables):
    """ Función para asignarle colores crecientes a una lista de elements

    Parámetros
    ----------
    variables: Lista de elementos a los cuales les queremos asignar color


    Regresa
    -------
    Dictionario de la forma: {element: color}
    """
    colors = plt.cm.jet(np.linspace(0, 1, len(variables)))
    return dict(zip(variables, colors))


def plot_numeric(df, numeric_stats):

    metrics = ['mean', 'median', 'std', 'q25', 'q75', 'nulls']
    colors = _get_colors_to_use(metrics)

    for index, variable in enumerate(sorted(numeric_stats.keys())):

        # Plotting basic metrics
        fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 5))

        bar_position = -1
        for metric, value in numeric_stats[variable].items():
            bar_position += 1

            if value is None or np.isnan(value):
                value = -1

            # Plotting bars
            bar_plot = ax[0].bar(bar_position, value,
                                 label=metric, color=colors[metric])
            autolabel(bar_plot, ax[0])

            # Plotting histogram
            df[variable].plot(kind='hist', color='blue',
                              alpha=0.4, ax=ax[1])

            # Plotting boxplot
            df.boxplot(ax=ax[2], column=variable)

            ax[0].set_xticks(range(len(metrics)))
            ax[0].set_xticklabels(metrics, rotation=90)
            ax[2].set_xticklabels('', rotation=90)

            ax[0].set_title('\n Basic metrics \n', fontsize=10)
            ax[1].set_title('\n Data histogram \n', fontsize=10)
            ax[2].set_title('\n Data boxplot \n', fontsize=10)
            fig.suptitle(f'Variable: {variable} \n\n\n', fontsize=15)

            fig.tight_layout()
    return


def plot_categorical(df, object_stats):

    metrics = ['unique_vals', 'mode', 'null_count']
    colors = _get_colors_to_use(metrics)

    for index, variable in enumerate(sorted(object_stats.keys())):

        # Plotting basic metrics
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

        bar_position = -1
        for metric, value in object_stats[variable].items():
            bar_position += 1

            if metric == 'mode':
                mode = value[0]
                value = value[1]

            if value is None or np.isnan(value):
                value = -1

            bar_plot = ax.bar(bar_position, value,
                              label=metric, color=colors[metric])
            autolabel(bar_plot, ax)

        ax.set_xticks(range(len(metrics)))
        ax.set_xticklabels(metrics, rotation=90, fontsize=15)

        ax.set_title(
            f'\n Basic object metrics: {variable} \n Mode: {mode}\n',
            fontsize=15)

        fig.tight_layout()
    return

def get_numeric_stats(df):
    """
    Esta magia sacará estadísticas básicas DE LAS VARIABLES NUMÉRICAS.

    Parámetros
    ----------
    df: pandas.DataFrame
        Tabla con variables limpias.

    Regresa
    -------
    stats: diccionario
        Dict de la forma {columna: {nombre de la métrica: valor de la métrica}}
    """
    # Seleccionando las variables numéricas únicamente
    numeric_df = df.select_dtypes(include=['float64', 'int64'])

    # Este va a ser el diccionario que regresaremos, lo llenaremos con un looop.
    stats = {}

    # Recorramos las columnas
    for numeric_column in numeric_df.columns:
        # Obtengamos el promedio
        mean = numeric_df[numeric_column].mean()

        # Ahora la mediana
        median = numeric_df[numeric_column].median()

        # Ahora la desviación estándar
        std = numeric_df[numeric_column].std()

        # Obtengamos el primer y tercer cuartil
        quantile25, quantile75 = numeric_df[numeric_column].quantile(
            q=[0.25, 0.75])

        # ¿Cuál es el porcentaje de nulos?
        null_count = 100 * (
        numeric_df[numeric_column].isnull().sum() / len(numeric_df))

        # Guardemos
        stats[numeric_column] = {'mean': mean,
                                 'median': median,
                                 'std': std,
                                 'q25': quantile25,
                                 'q75': quantile75,
                                 'nulls': null_count
                                 }
    return stats

def get_cat_stats(df):
    """
    Esta magia sacará estadísticas básicas DE LAS VARIABLES CATEGÓRICAS

    Parámetros
    ----------
    df: pandas.DataFrame
        Tabla con variables limpias.

    Regresa
    -------
    stats: diccionario
        Dict de la forma {columna: {nombre de la métrica: valor de la métrica}}
    """
    # Seleccionando los objetos
    object_df = df.select_dtypes(include=['object', 'category'])

    # El dict que regresaremos
    stats = {}

    # Recorramos las columnas
    for object_column in object_df.columns:
        # ¿Cuántos valores únicos hay?
        unique_vals = len(object_df[object_column].unique())

        # Saquemos la "moda" (valor más común).
        # Para eso primero usamos value_counts para encontrar la frecuenc
        all_values = object_df[object_column].value_counts()

        # Ahora sacaremos una tupla con el valor más común y el porcentaje de veces
        # que aparece
        mode = (all_values.index[0],
                100 * (all_values.values[0] / len(object_df)))

        # Cuenta de nulos
        null_count = (object_df[object_column].isnull().sum() / len(
            object_df)) * 100

        # Stats a devolver
        stats[object_column] = {'unique_vals': unique_vals,
                                'mode': mode,
                                'null_count': null_count}

    return stats

In [8]:
# Seleccionar columnas que sean numéricas
numeric_cols = df.select_dtypes(include=['int64', 'float64'])
numeric_df = df[numeric_cols.columns]

In [ ]:
#Variables numéricas.
numeric_stats = get_numeric_stats(numeric_df)   
plot_numeric(df, numeric_stats)

En las gráficas de estadísticas básicas, podemos ver lo siguiente:
+ `dew`: No tiene datos atípicos y su distribución no está muy anormal
+ `pollution`: Contiene varios datos atípicos, su promedio es de 94 y los valores suelen acotarse entre 24 y 132
+ `press`: No hay datos atípicos y la distribución parece acercarse a la normal, los valores son muy similares entre sí
+ `rain`: El promedio es de 0 y hay bastantes atípicos
+ `snow`: Al igual que la variable de lluvia, su promedio es 0 y hay muchos atípicos, esto no debe alarmar ya que son las horas acumuladas de nieve/lluvia
+ `temp`: No hay atípicos y su distribución parece ser normal, el promedio es de 14
+ `wnd_spd`: Hay bastantes atípicos y la desviación estándar es muy alta, igual no hay que alarmarse ya que es la velocidad del viento acumulada

In [ ]:
#Variables categóricas.
object_cols = df.select_dtypes(include=['object', 'category'])
object_df = object_cols.loc[:, ~object_cols.columns.str.contains('date', case=False)]
cat_stats = get_cat_stats(object_df)
plot_categorical(df, cat_stats)

Aquí podemos ver que la variable `wnd_dir` tiene 4 posibles valores y la moda es SE, indicando South-East.

In [11]:
df_codificado = df.copy()

In [12]:
def label_encode_variables_categoricas(df_codificado):
    # Selecciona las columnas de tipo objeto (categóricas)
    columnas_objeto = df_codificado.select_dtypes(include=['object']).columns

    # Inicializa el codificador
    label_encoder = LabelEncoder()

    # Itera sobre las columnas seleccionadas y aplica el label encoding
    for columna in columnas_objeto:
        df_codificado[columna] = label_encoder.fit_transform(df_codificado[columna])

    return df_codificado

df_para_corr = label_encode_variables_categoricas(df_codificado)

In [ ]:
# Calcula la matriz de correlación excluyendo la columna date
matriz_correlacion = df_para_corr.drop('date', axis=1).corr()

# Configura el tamaño de la figura
plt.figure(figsize=(8, 8))

# Genera la matriz de correlación con Seaborn
sns.heatmap(matriz_correlacion, annot=True, cmap='coolwarm', fmt=".2f")

# Añade título
plt.title('Matriz de Correlación')

# Muestra la matriz de correlación
plt.show()

Matriz de correlaciones:
1. Relaciones positivas fuertes
    + `temp` y `dew` tienen una correlación positiva de 0.82
2. Relaciones negativas fuertes
    + `press` y `dew` tienen una correlación negativa de -0.78, `press` y `temp` de -0.83

Las demás variables no muestran fuerte correlación con la siguiente más correlacionada siendo `dew` y `wnd_spd` con -0.30 

#### Análisis univariado para SARIMAX

In [ ]:
# Convert date column to datetime if it isn't already
df['date'] = pd.to_datetime(df['date'])

# Create time series with date as index and pollution as values
time_series = df.set_index('date')['pollution']
time_series = pd.DataFrame(time_series)
time_series = time_series.rename(columns={'pollution': 'y'})
time_series.head()

In [ ]:
time_series.shape

In [ ]:
# Plot the time series
plt.figure(figsize=(15, 8))
plt.plot(time_series.index, time_series['y'])
plt.title('Time Series of Pollution')
plt.xlabel('Date')
plt.ylabel('Pollution')
plt.grid()
plt.show()

In [ ]:
# Vamos a graficar el boxplot
time_series.boxplot()

In [ ]:
# Detectar outliers con IQR
Q1 = time_series['y'].quantile(0.25)
Q3 = time_series['y'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = time_series[(time_series['y'] < lower_bound) | (time_series['y'] > upper_bound)]
print("Outliers detected:")
print(outliers)

In [ ]:
reporte(outliers)

In [ ]:
# Filter time series for 2010-2011
time_series_2010_2011 = time_series['2010-02':'2010-03']

# Plot the filtered time series
plt.figure(figsize=(15, 8))
plt.plot(time_series_2010_2011.index, time_series_2010_2011['y'])
plt.title('Time Series of Pollution February-March 2010')
plt.xlabel('Date') 
plt.ylabel('Pollution')
plt.grid()
plt.show()


Al graficar y analizar la variable de interés, nos encontramos con varios datos atípicos, primero en el boxplot y después con el método del rango intercuartil. De los 43,800 registros que tenemos, nos marca 1,878 atípicos. Al ver una muestra de los que marca como atípicos, vemos valores superiores a 300. Al usar nuestra función de reporte, vemos que alcanzan máximos de hasta 994.

Después de una breve investigación, encontramos que la página oficial de calidad del aire de Estados Unidos, AirNow, tiene los siguientes rangos de calidad:

![Air Quality Table](images/Air_pollution_table.png)

Con esta información, podría parecer que son errores de medición, sin embargo, valores de más de 300 pueden ser totalmentes posibles, ya que graficamos una parte en donde se encuentran estos valores atípicos, y parece que los días siguientes al atípico siguen siendo valores grandes, entonces no son ni errores de medición ni imposibles. Es por esto que decidimos dejarlos.

AQI Basics | AirNow.gov. (n.d.). https://www.airnow.gov/aqi/aqi-basics/

In [21]:
# Dividir los datos en entrenamiento y prueba
n_train = int(0.75 * len(time_series))
train_data = time_series[:n_train]
n_test = len(time_series) - n_train
test_data = time_series[-n_test:]

In [22]:
lf = LinearForecast(data=train_data)

In [ ]:
lf.adf_test() # Estacionariedad de los datos originales
# Vamos a visualizar todas las transformaciones para luego decidir cuál será la elegida

In [ ]:
lf.timeseries_transformation(transformation="sqrt")

In [ ]:
lf.timeseries_transformation(transformation="log")

In [ ]:
lf.timeseries_transformation(transformation="yeo-johnson")

In [ ]:
train, lambda_value = lf.consolidated_ts_df(data=train_data, plot_graphs=True)
train.head()

Después de ver los resultados de las transformaciones, elegiremos la de yeo-johnson. La segunda mejor fue la de log, sin embargo, como hay valores que son 0 en nuestra serie (misma razón por la cuál no aplicó box-cox), la de yeo-johnson es mejor.

In [28]:
train.drop([col for col in train.columns if 'yeo_j' not in col], axis=1, inplace=True)
train.rename(columns={'yeo_j': 'y'}, inplace=True)

In [29]:
lf = LinearForecast(data=train)

In [ ]:
lf.plot_time_series()

Ahora evaluaremos la estacionariedad con la prueba de ADF y la descomposición para analizar los componentes.

In [ ]:
lf.adf_test()

In [32]:
def mstl_descomposition1(data, periods_seasonality, stl_kwargs=None):
    """
    Perform MSTL decomposition to identify trend, seasonal, and residual components of a time series.

    Parameters:
        periods_seasonality (list): List of seasonal periods to be considered.
        stl_kwargs (dict, optional): Additional keyword arguments for STL decomposition.

    Returns:
        DecomposeResult: A result object that includes trend, seasonal components, and residuals.
    """
    if stl_kwargs is None:
        stl_kwargs = {}

    if data is None or data.empty:
        raise ValueError("The data parameter is empty or None.")

    y = data['y']
    mstl = MSTL(endog=y, periods=periods_seasonality, **stl_kwargs)
    result = mstl.fit()

    return result

In [ ]:
res = mstl_descomposition1(train,periods_seasonality=[24, 168]) # 24 por ser diario, 168 semanal
res.plot().set_size_inches(12, 8)
plt.show()

In [ ]:
# Graficar datos observados con datos de estacionalidad
ax = res.observed[:400].plot(label='observados')

res.seasonal['seasonal_24'][:400].plot(ax=ax, label='seasonal 24')
res.seasonal['seasonal_168'][:400].plot(ax=ax, label='seasonal 168')

plt.legend()
plt.show()

##### Estacionariedad de la estacionalidad encontrada

In [ ]:
# En base a la estacionalidad seleccionada, calcular la estacionariedad de dicha serie temporal
seasonality_24 = res.seasonal['seasonal_24']

lf.adf_test(seasonality_24)

In [ ]:
# Estacionariedad de la segunda parte estacional
seasonality_168 = res.seasonal['seasonal_168']

lf.adf_test(seasonality_168)

In [37]:
trend_data = res.trend
resid_data = res.resid

In [ ]:
lf.adf_test(trend_data) # Estacionariedad de la tendencia

In [ ]:
lf.adf_test(resid_data) # Estacionariedad de residuos

##### Genere los correlogramas ACF y PACF para evaluar patrones de autocorrelación.

In [ ]:
lf.plot_acf_pacf()

In [ ]:
lf.plot_acf_pacf(data=train.diff().diff(24).dropna())

In [ ]:
lf.plot_acf_pacf(data=train.diff().diff(168).dropna())

Basado en las gráficas, los posibles valores son:

P -> 1

D -> 1

Q -> 1

Ahora para el componenete no estacional

In [ ]:
# Calcular la estacionariedad de mis datos observados
lf.adf_test(res.observed.dropna())

In [ ]:
# Calcular la componente no estacional de la serie temporal basado en la S anterior
non_seasonal = res.observed - res.seasonal['seasonal_24'] - res.seasonal['seasonal_168'] 

res.observed[:200].plot()
non_seasonal[:200].plot()
plt.legend(['observed', 'non-seasonal'])

In [ ]:
# Calcular si la serie no estacional es estacionaria o no
lf.adf_test(non_seasonal.dropna())

In [ ]:
lf.plot_acf_pacf(non_seasonal.dropna())

Para los posibles valores son:

p -> 2

d -> 0

q -> 0

### SARIMAX

In [47]:
from sklearn.preprocessing import PowerTransformer

pt = PowerTransformer(method='yeo-johnson')
# Transformar los datos de entrenamiento y prueba
train_transformed = pt.fit_transform(train_data.values.reshape(-1, 1))
test_transformed = pt.transform(test_data.values.reshape(-1, 1))

Vamos a loggear el experimento en MLFlow.

In [ ]:
dagshub.init(url="https://dagshub.com/daduke1/proyecto-modelos", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="proyecto-modelos")

In [ ]:
order = (2, 0, 0)
seasonal_order = (1, 1, 1, 24)

with mlflow.start_run() as run:
    # Log parameters
    mlflow.log_param("order", order)
    mlflow.log_param("seasonal_order", seasonal_order)

    # Fit model
    model = SARIMAX(train_transformed, order=order, seasonal_order=seasonal_order,
                    enforce_stationarity=False, enforce_invertibility=False)
    results = model.fit(disp=False)

    # Save model
    model_file = "sarimax_model.pkl"
    joblib.dump(results, model_file)
    mlflow.log_artifact(model_file)

    # Forecast
    forecast_trans = ph_model.forecast(steps=len(test_transformed))
    forecast_trans = pd.DataFrame(forecast_trans)
    forecast_inv = pt.inverse_transform(forecast_trans).flatten()

    # Compute and log metrics
    rmse = np.sqrt(mean_squared_error(test_data, forecast_inv))
    mse = mean_squared_error(test_data, forecast_inv)
    mape = mean_absolute_percentage_error(test_data, forecast_inv)
    r2 = r2_score(test_data, forecast_inv)

    mlflow.log_metric("RMSE_test", rmse)
    mlflow.log_metric("MSE_test", mse)
    mlflow.log_metric("MAPE_test", mape)
    mlflow.log_metric("R2_test", r2)

    # Log model summary
    summary_file = "model_summary.txt"
    with open(summary_file, "w") as f:
        f.write(results.summary().as_text())
    mlflow.log_artifact(summary_file)

    # Plot forecast vs actual
    plt.figure(figsize=(10, 6))
    plt.plot(test_data.index, test_inv, label="Actual")
    plt.plot(test_data.index, forecast_inv, label="Forecast", linestyle='--')
    plt.title("SARIMAX Forecast vs Actual (Test Set)")
    plt.legend()
    plt.tight_layout()
    plot_file = "forecast_vs_actual.png"
    plt.savefig(plot_file)
    plt.close()
    mlflow.log_artifact(plot_file)

    print(f"Logged SARIMAX model. RMSE: {rmse:.2f}, MAPE: {mape:.2%}, R²: {r2:.3f}")

El modelo SARIMAX elegido dió buenos resultados, con las siguientes métricas:

+ MSE: 0.11
+ MAPE: 77.79%
+ R2: 0.894